# BLIP finetuning to target task

TODO: rewrite this to reflect the latest changes

Sample is formed from a single row of dataset:
$$\operatorname{batch} = ((E_t, E_{i_0}), (E_t, E_{i_1}), ..., (E_t, E_{i_9})); \operatorname{batch} : R^{10 \times (E_t + E_i)}$$
ITM predicts probas for $y = 0$, $y = 1$
$$\operatorname{ITM} : R^{10 \times (E_t + E_i)} \rightarrow R^{10 \times 2}$$
Model is defined as:
$$\operatorname{F} = \operatorname{softmax} \circ \operatorname{ITM} \circ \operatorname{batch}$$
$$\operatorname{F} : R^{10 \times (E_t + E_i)} \rightarrow R^{10}$$
So, this definition is for a single row

In [1]:
from pathlib import Path
import logging
import json
from typing import *
import time
from datetime import datetime

import pandas as pd
import numpy as np
import torch
from PIL import Image, ImageFile
import torch.nn as nn
from lavis.models import load_model_and_preprocess, BlipBase
from lavis.processors import load_processor
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup
from transformers import BatchEncoding
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import top_k_accuracy_score

from src.data import CustomSplitLoader
from src.utils import evaluate, mrr
from src.itc import ClsITC, ClsITCBatchData, Temperature

## Config

Versioning

In [2]:
HEAD = "itc" # only this value is possible in this notebook
MODEL_VERSION = 5

Paths resolution:

In [3]:
DATASET_VERSION = "v1"
PART = "train"
PATH = Path("data").resolve() / f"{PART}_{DATASET_VERSION}"
DATA_PATH = PATH / f"{PART}.data.{DATASET_VERSION}.txt"
LABELS_PATH = PATH / f"{PART}.gold.{DATASET_VERSION}.txt"
IMAGES_PATH = PATH / f"{PART}_images_{DATASET_VERSION}"
TRAIN_SPLIT_PATH = PATH / "split_train.txt"
VALIDATION_SPLIT_PATH = PATH / "split_valid.txt"
VAL2_DATA_PATH = PATH / "valid2.data.v1.txt"
VAL2_GOLD_PATH = PATH / "valid2.gold.v1.txt"
TEST_SPLIT_PATH = PATH / "split_test.txt"
TEST2_DATA_PATH = PATH / "test2.data.v1.txt"
TEST2_GOLD_PATH = PATH / "test2.gold.v1.txt"
SAVE_CHECKPOINT_PATH = Path("checkpoints").resolve() / f"BLIP-{HEAD}-{MODEL_VERSION}"
SAVE_CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

Environment settings:

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# some images from train might not load without the following settings or warnings would be thrown
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

writer = SummaryWriter(f"runs/blip-{HEAD}-{MODEL_VERSION} (ran at {datetime.now()})")

In [5]:
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
# WARNING: this is specific to my setup
# DEVICE = torch.device("cuda:0")
DEVICE = torch.device("cuda:1")
# a more conventional way to do this is:
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 32
print(f"Running on {DEVICE}")

Running on cuda:1


Model & training settings

In [6]:
BLIP_VARIANT = "base" # "base" | "large"
NUM_EPOCHS = 40
NUM_PICS = 10
WARMUP_STEPS_FRAC = 0.1
GRAD_ACCUM_STEPS = 24
LR = 5e-5
WEIGHT_DECAY = 0.001
TRAIN_BATCH_SIZE = 1

In [7]:
TRAIN_EFFECTIVE_BATCH_SIZE = GRAD_ACCUM_STEPS * TRAIN_BATCH_SIZE
TRAIN_EFFECTIVE_BATCH_SIZE

24

In [8]:
STEPS_BETWEEN_VAL = 250
STEPS_BETWEEN_VAL2 = 500
SAVE_CHECKPOINT_STEPS = STEPS_BETWEEN_VAL
VALIDATION_BATCH_SIZE = 4

## Loading data

In [9]:
df = pd.read_csv(DATA_PATH, sep='\t', header=None)
df.columns = ["word", "context"] + [f"image{i}" for i in range(NUM_PICS)]
df["label"] = pd.read_csv(LABELS_PATH, sep='\t', header=None)

train_df = df.loc[pd.read_csv(TRAIN_SPLIT_PATH, sep='\t', header=None).T.values[0]]
validation_df = df.loc[pd.read_csv(VALIDATION_SPLIT_PATH, sep='\t', header=None).T.values[0]]
test_df = df.loc[pd.read_csv(TEST_SPLIT_PATH, sep='\t', header=None).T.values[0]]

val2_df = pd.read_csv(VAL2_DATA_PATH, sep = '\t', header = None)
val2_df.columns = ["word", "context"] + [f"image{i}" for i in range(10)]
val2_df["label"] = pd.read_csv(VAL2_GOLD_PATH, sep = "\t", header = None)

test2_df = pd.read_csv(TEST2_DATA_PATH, sep = '\t', header = None)
test2_df.columns = ["word", "context"] + [f"image{i}" for i in range(10)]
test2_df["label"] = pd.read_csv(TEST2_GOLD_PATH, sep = "\t", header = None)

In [10]:
train_df

,word,context,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,label
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg
5,ixia,ixia genus,image.90.jpg,image.3.jpg,image.91.jpg,image.4.jpg,image.92.jpg,image.1.jpg,image.2.jpg,image.94.jpg,image.93.jpg,image.5.jpg,image.90.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12861,ducking,ducking hunting,image.964.jpg,image.6176.jpg,image.6742.jpg,image.12919.jpg,image.9996.jpg,image.966.jpg,image.967.jpg,image.12662.jpg,image.4312.jpg,image.965.jpg,image.12919.jpg
12862,tarnish,tarnish discoloration,image.7862.jpg,image.11086.jpg,image.11714.jpg,image.5269.jpg,image.2789.jpg,image.11230.jpg,image.3341.jpg,image.224.jpg,image.222.jpg,image.220.jpg,image.11714.jpg
12865,tragopogon,tragopogon genus,image.3.jpg,image.6250.jpg,image.15001.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.12074.jpg,image.5.jpg,image.4087.jpg,image.12806.jpg,image.12074.jpg
12866,illustrator,illustrator artist,image.10633.jpg,image.723.jpg,image.13372.jpg,image.881.jpg,image.12635.jpg,image.726.jpg,image.5985.jpg,image.722.jpg,image.724.jpg,image.725.jpg,image.10633.jpg


In [11]:
validation_df

,word,context,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,label
18,maja,maja genus,image.3.jpg,image.310.jpg,image.4.jpg,image.309.jpg,image.1.jpg,image.2.jpg,image.312.jpg,image.5.jpg,image.311.jpg,image.313.jpg,image.309.jpg
24,entoloma,entoloma genus,image.3.jpg,image.405.jpg,image.404.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.406.jpg,image.407.jpg,image.408.jpg,image.404.jpg
25,foulard,foulard fabric,image.340.jpg,image.418.jpg,image.423.jpg,image.343.jpg,image.421.jpg,image.344.jpg,image.422.jpg,image.342.jpg,image.208.jpg,image.420.jpg,image.418.jpg
27,biryani,biryani dish,image.454.jpg,image.436.jpg,image.455.jpg,image.453.jpg,image.451.jpg,image.456.jpg,image.437.jpg,image.457.jpg,image.434.jpg,image.433.jpg,image.451.jpg
28,sobriquet,sobriquet appellation,image.466.jpg,image.478.jpg,image.477.jpg,image.475.jpg,image.476.jpg,image.471.jpg,image.474.jpg,image.473.jpg,image.472.jpg,image.329.jpg,image.466.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12851,marattia,marattia genus,image.3.jpg,image.11008.jpg,image.6217.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.11414.jpg,image.5.jpg,image.9223.jpg,image.9977.jpg,image.6217.jpg
12852,tragulus,tragulus genus,image.3.jpg,image.10621.jpg,image.8594.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.4061.jpg,image.14197.jpg,image.12606.jpg,image.4061.jpg
12853,barbwire,barbwire wire,image.58.jpg,image.59.jpg,image.57.jpg,image.5849.jpg,image.56.jpg,image.4112.jpg,image.2784.jpg,image.151.jpg,image.60.jpg,image.6659.jpg,image.4112.jpg
12855,sample,sample distribution,image.65.jpg,image.13362.jpg,image.3623.jpg,image.6254.jpg,image.12852.jpg,image.290.jpg,image.10966.jpg,image.3473.jpg,image.3474.jpg,image.3472.jpg,image.10966.jpg


In [12]:
test_df

,word,context,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,label
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg,image.55.jpg
6,leucaena,leucaena genus,image.105.jpg,image.3.jpg,image.106.jpg,image.109.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.108.jpg,image.5.jpg,image.107.jpg,image.105.jpg
7,mahonia,mahonia genus,image.3.jpg,image.124.jpg,image.122.jpg,image.4.jpg,image.120.jpg,image.123.jpg,image.1.jpg,image.2.jpg,image.121.jpg,image.5.jpg,image.120.jpg
10,gangster,gangster outlaw,image.166.jpg,image.173.jpg,image.172.jpg,image.165.jpg,image.174.jpg,image.170.jpg,image.171.jpg,image.167.jpg,image.168.jpg,image.169.jpg,image.165.jpg
12,brevicipitidae,brevicipitidae family,image.3.jpg,image.207.jpg,image.206.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.205.jpg,image.208.jpg,image.209.jpg,image.205.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,make,make persuade,image.442.jpg,image.9126.jpg,image.7574.jpg,image.7582.jpg,image.5015.jpg,image.5704.jpg,image.4933.jpg,image.1022.jpg,image.2288.jpg,image.1208.jpg,image.9126.jpg
12856,gunboat,gunboat boat,image.615.jpg,image.364.jpg,image.58.jpg,image.59.jpg,image.122.jpg,image.57.jpg,image.4149.jpg,image.56.jpg,image.680.jpg,image.60.jpg,image.615.jpg
12857,francisella,francisella bacteria,image.3.jpg,image.4.jpg,image.1147.jpg,image.2798.jpg,image.1.jpg,image.2.jpg,image.14303.jpg,image.5.jpg,image.8422.jpg,image.4973.jpg,image.1147.jpg
12863,lookout,lookout watcher,image.5338.jpg,image.11952.jpg,image.58.jpg,image.59.jpg,image.57.jpg,image.56.jpg,image.10445.jpg,image.15132.jpg,image.4060.jpg,image.60.jpg,image.4060.jpg


In [13]:
val2_df

,word,context,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,label
0,maja,maja genus,image.309.jpg,image.2499.jpg,image.9528.jpg,image.3312.jpg,image.11360.jpg,image.7933.jpg,image.13433.jpg,image.3402.jpg,image.1488.jpg,image.1206.jpg,image.309.jpg
1,sobriquet,sobriquet appellation,image.1402.jpg,image.10671.jpg,image.12672.jpg,image.7395.jpg,image.10488.jpg,image.1341.jpg,image.466.jpg,image.3816.jpg,image.6705.jpg,image.8732.jpg,image.466.jpg
2,pigiron,pigiron iron,image.485.jpg,image.3312.jpg,image.2829.jpg,image.6694.jpg,image.627.jpg,image.1724.jpg,image.5215.jpg,image.9603.jpg,image.10601.jpg,image.7395.jpg,image.485.jpg
3,paddle,paddle beat,image.2626.jpg,image.432.jpg,image.545.jpg,image.3482.jpg,image.2708.jpg,image.373.jpg,image.16015.jpg,image.9314.jpg,image.4231.jpg,image.7411.jpg,image.545.jpg
4,gourmand,gourmand feeder,image.2443.jpg,image.7923.jpg,image.9567.jpg,image.15660.jpg,image.634.jpg,image.9426.jpg,image.6064.jpg,image.4139.jpg,image.6676.jpg,image.1964.jpg,image.634.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,caco3,caco3 carbonate,image.8703.jpg,image.2859.jpg,image.14885.jpg,image.10018.jpg,image.11685.jpg,image.271.jpg,image.15113.jpg,image.8715.jpg,image.8338.jpg,image.15640.jpg,image.8338.jpg
1551,marattia,marattia genus,image.1596.jpg,image.9510.jpg,image.6363.jpg,image.1122.jpg,image.1647.jpg,image.5751.jpg,image.3599.jpg,image.2078.jpg,image.6217.jpg,image.7841.jpg,image.6217.jpg
1552,tragulus,tragulus genus,image.7219.jpg,image.4252.jpg,image.12298.jpg,image.4061.jpg,image.13118.jpg,image.14801.jpg,image.6824.jpg,image.7813.jpg,image.9131.jpg,image.9338.jpg,image.4061.jpg
1553,barbwire,barbwire wire,image.10115.jpg,image.4112.jpg,image.11677.jpg,image.5307.jpg,image.498.jpg,image.11176.jpg,image.9644.jpg,image.2320.jpg,image.9184.jpg,image.6242.jpg,image.4112.jpg


In [14]:
test2_df

,word,context,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,label
0,leucaena,leucaena genus,image.11561.jpg,image.9445.jpg,image.10639.jpg,image.3186.jpg,image.9619.jpg,image.105.jpg,image.8520.jpg,image.14730.jpg,image.14089.jpg,image.13274.jpg,image.105.jpg
1,mahonia,mahonia genus,image.1995.jpg,image.120.jpg,image.385.jpg,image.605.jpg,image.1993.jpg,image.3456.jpg,image.4563.jpg,image.2947.jpg,image.8532.jpg,image.3763.jpg,image.120.jpg
2,breakdown,breakdown failure,image.5190.jpg,image.13274.jpg,image.2285.jpg,image.5666.jpg,image.239.jpg,image.6912.jpg,image.7786.jpg,image.9475.jpg,image.13051.jpg,image.8279.jpg,image.239.jpg
3,boletellus,boletellus genus,image.2193.jpg,image.324.jpg,image.8584.jpg,image.7562.jpg,image.7509.jpg,image.14485.jpg,image.13328.jpg,image.4791.jpg,image.13053.jpg,image.10828.jpg,image.324.jpg
4,capparis,capparis genus,image.15880.jpg,image.7367.jpg,image.8603.jpg,image.359.jpg,image.10262.jpg,image.116.jpg,image.2622.jpg,image.2896.jpg,image.481.jpg,image.7507.jpg,image.359.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554,aspergillus,aspergillus genus,image.7847.jpg,image.5827.jpg,image.7330.jpg,image.1632.jpg,image.7323.jpg,image.10614.jpg,image.2062.jpg,image.12440.jpg,image.9796.jpg,image.5521.jpg,image.10614.jpg
1555,mantophasmatodea,mantophasmatodea order,image.777.jpg,image.13482.jpg,image.1892.jpg,image.14892.jpg,image.6142.jpg,image.13245.jpg,image.3355.jpg,image.5598.jpg,image.2285.jpg,image.3602.jpg,image.3355.jpg
1556,make,make persuade,image.12886.jpg,image.1263.jpg,image.9126.jpg,image.9526.jpg,image.12047.jpg,image.2597.jpg,image.9154.jpg,image.8859.jpg,image.14239.jpg,image.6168.jpg,image.9126.jpg
1557,lookout,lookout watcher,image.9975.jpg,image.593.jpg,image.4049.jpg,image.4060.jpg,image.2628.jpg,image.7152.jpg,image.11177.jpg,image.9415.jpg,image.6207.jpg,image.10997.jpg,image.4060.jpg


## Preprocessing

In [15]:
def infinite_repeat(value):
    while True:
        yield value

def concat_iters(*iterables):
    for it in iterables:
        for value in it:
            yield value

class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        images_path: Path,
        text_processor,
        vis_processor,
        use_context_as_text: bool = True,
    ) -> None:
        self.df = df
        self.images_path = images_path
        self.text_processor = text_processor
        self.vis_processor = vis_processor
        self.text_field = "context" if use_context_as_text else "word"
        self.labels_map = self._gen_labels()

    def _gen_labels(self) -> Dict[int, int]: # index to label
        labels = self.df["label"].values
        zips = []
        for i in range(NUM_PICS):
            images = self.df[f"image{i}"].values
            zips.append(zip(np.argwhere(labels == images).reshape(-1), infinite_repeat(i)))
        return dict(concat_iters(*tuple(zips)))
    
    def __len__(self) -> int:
        return len(self.df)

    def _make_image_tensor(self, name: str) -> torch.Tensor:
        return self.vis_processor(Image.open(self.images_path / name).convert("RGB"))
    
    def _make_image_batch(self, idx: int) -> torch.Tensor:
        row = self.df.iloc[idx]
        return torch.stack([self._make_image_tensor(row[f"image{i}"]) for i in range(NUM_PICS)])

    def _make_tokens(self, idx: int) -> BatchEncoding:
        return self.text_processor(self.df.iloc[idx][self.text_field])

    def __getitem__(self, idx: int) -> Dict[str, Union[torch.Tensor, BatchEncoding, int]]:
        # makes a batch for each row!
        return {
            "text": self._make_tokens(idx),
            "images": self._make_image_batch(idx),
            "label": self.labels_map[idx],
        }


def collate_fn(lst: List[Dict]) -> Tuple[ClsITCBatchData, torch.Tensor]:
    """
    Custom collator for ClsITC model

    Args:
        lst (List[Dict]): list of dictionaries made by Dataset instance

    Returns:
        Tuple[ClsITCBatchData, torch.Tensor]: batch data for the model & target tensor
    """
    image_sets = []
    texts = []
    labels = []
    for item in lst:
        image_sets.append(item["images"])
        texts.append(item["text"])
        labels.append(item["label"])
    return ClsITCBatchData(
        images=torch.stack(image_sets),
        text=texts,
    ), torch.tensor(labels).long()


In [16]:
blip_model, vis_processors, text_processors = load_model_and_preprocess("blip_image_text_matching", BLIP_VARIANT, is_eval=True)

INFO:root:Missing keys []
INFO:root:load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth


In [17]:
train_ds = Dataset(
    df=train_df,
    images_path=IMAGES_PATH,
    text_processor=text_processors["eval"],
    vis_processor=vis_processors["eval"],
)
val_ds = Dataset(
    df=validation_df,
    images_path=IMAGES_PATH,
    text_processor=text_processors["eval"],
    vis_processor=vis_processors["eval"],
)
val2_ds = Dataset(
    df=val2_df,
    images_path=IMAGES_PATH,
    text_processor=text_processors["eval"],
    vis_processor=vis_processors["eval"],
)
test_ds = Dataset(
    df=test_df,
    images_path=IMAGES_PATH,
    text_processor=text_processors["eval"],
    vis_processor=vis_processors["eval"],
)
test2_ds = Dataset(
    df=test2_df,
    images_path=IMAGES_PATH,
    text_processor=text_processors["eval"],
    vis_processor=vis_processors["eval"],
)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, persistent_workers=True, collate_fn=collate_fn)
train_l = len(train_dl)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=VALIDATION_BATCH_SIZE, num_workers=NUM_WORKERS, persistent_workers=True, collate_fn=collate_fn)
val_l = len(val_dl)
val2_dl = torch.utils.data.DataLoader(val2_ds, batch_size=VALIDATION_BATCH_SIZE, num_workers=NUM_WORKERS, persistent_workers=True, collate_fn=collate_fn)
val2_l = len(val2_dl)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, num_workers=NUM_WORKERS, persistent_workers=True, shuffle=False, collate_fn=collate_fn)
test_l = len(test_dl)
test2_dl = torch.utils.data.DataLoader(test2_ds, batch_size=1, num_workers=NUM_WORKERS, persistent_workers=True, shuffle=False, collate_fn=collate_fn)
test2_l = len(test2_dl)

train_l, val_l, val2_l, test_l, test2_l

(6103, 853, 389, 3356, 1559)

## Model setup

In [18]:
model = ClsITC(
    blip_itm = blip_model,
    tokenizer_settings = None, # defaults
    temperature = Temperature(5),
    apply_softmax = True, # apply softmax on outputs of cosine-similarities
).to(DEVICE)

## Training

In [19]:
metric2name = {
    "acc1": "Accuracy@Top1",
    "acc3": "Accuracy@Top3",
    "mrr": "Mean Reciprocal Rank",
}

labels_range = np.arange(NUM_PICS)

def eval_batch(labels, preds):
    labels = labels.numpy(force=True)
    preds = preds.numpy(force=True)
    return {
        "acc1": top_k_accuracy_score(labels, preds, k=1, labels=labels_range), 
        "acc3": top_k_accuracy_score(labels, preds, k=3, labels=labels_range),
        "mrr": mrr(labels, preds),
    }

def sum_scores(scores, new_scores):
    return {k: scores[k] + new_scores[k] for k in scores}

def div_scores(scores, n):
    return {k: v / n for k, v in scores.items()}

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
num_training_steps = int(NUM_EPOCHS * (train_l / GRAD_ACCUM_STEPS))
num_warmup_steps = int(num_training_steps * WARMUP_STEPS_FRAC)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)
print(f"{num_training_steps} training steps which include {num_warmup_steps} warmup ones")

10171 training steps which include 1017 warmup ones


In [21]:
step_num = 0
steps_since_last_val = 0
steps_since_last_val2 = 0
grad_accum_step_cnt = 0
save_checkpoint_step_cnt = 0
progress_bar = tqdm(range(num_training_steps))

for epoch_num in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_scores = {"acc1": 0, "acc3": 0, "mrr": 0}
    for data, labels in train_dl:
        outputs = model(data.to(DEVICE))
        loss = loss_fn(outputs, F.one_hot(labels, NUM_PICS).float().to(DEVICE))

        train_loss += loss.item()
        train_scores = sum_scores(train_scores, eval_batch(labels, outputs))

        loss.backward()
        grad_accum_step_cnt += 1

        if grad_accum_step_cnt == GRAD_ACCUM_STEPS: 
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            writer.add_scalar("Loss/Train", train_loss / GRAD_ACCUM_STEPS, step_num)            
            for k, v in div_scores(train_scores, GRAD_ACCUM_STEPS).items():
                writer.add_scalar(metric2name[k] + "/Train", v, step_num)
            train_loss = 0.0
            train_scores = {"acc1": 0, "acc3": 0, "mrr": 0}
            grad_accum_step_cnt = 0
            step_num += 1
            steps_since_last_val += 1
            steps_since_last_val2 += 1
            save_checkpoint_step_cnt += 1
            progress_bar.update(1)

        if steps_since_last_val == STEPS_BETWEEN_VAL:
            model.eval()
            val_loss = 0.0
            val_scores = {"acc1": 0, "acc3": 0, "mrr": 0}
            with torch.no_grad():
                for data, labels in val_dl:
                    outputs = model(data.to(DEVICE))
                    loss = loss_fn(outputs, F.one_hot(labels, NUM_PICS).float().to(DEVICE))
                    val_loss += loss.item()
                    val_scores = sum_scores(val_scores, eval_batch(labels, outputs))
            writer.add_scalar("Loss/Validation", val_loss / val_l, step_num)            
            for k, v in div_scores(val_scores, val_l).items():
                writer.add_scalar(metric2name[k] + "/Validation", v, step_num)
            model.train()
            steps_since_last_val = 0
        
        if steps_since_last_val2 == STEPS_BETWEEN_VAL2:
            model.eval()
            val_loss = 0.0
            val_scores = {"acc1": 0, "acc3": 0, "mrr": 0}
            with torch.no_grad():
                for data, labels in val2_dl:
                    outputs = model(data.to(DEVICE))
                    loss = loss_fn(outputs, F.one_hot(labels, NUM_PICS).float().to(DEVICE))
                    val_loss += loss.item()
                    val_scores = sum_scores(val_scores, eval_batch(labels, outputs))
            writer.add_scalar("Loss/Validation 2", val_loss / val2_l, step_num)            
            for k, v in div_scores(val_scores, val2_l).items():
                writer.add_scalar(metric2name[k] + "/Validation 2", v, step_num)
            model.train()
            steps_since_last_val2 = 0
        
        if save_checkpoint_step_cnt == SAVE_CHECKPOINT_STEPS:
            save_checkpoint_step_cnt = 0
            p = SAVE_CHECKPOINT_PATH / f"step-{step_num}.pt"
            torch.save(model.state_dict(), p)

  1%|          | 92/10171 [23:29<42:34:15, 15.21s/it]/home/s1m00n/miniconda3/envs/lavis/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
  1%|          | 105/10171 [26:48<42:57:42, 15.36s/it]/home/s1m00n/miniconda3/envs/lavis/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
  1%|          | 108/10171 [27:33<42:38:23, 15.25s/it]/home/s1m00n/miniconda3/envs/lavis/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
  1%|          | 125/10171 [31:52<42:26:21, 15.21s/it]

## Evaluation

Here, let's load the best checkpoint according to Tensorboard

In [ ]:
# TODO, copy from eval-test2.ipynb

In [ ]:
CHECKPOINT_NUM = 6100

In [ ]:
checkpoint = Classifier(blip_model).to(DEVICE)
checkpoint.load_state_dict(torch.load(SAVE_CHECKPOINT_PATH / f"step-{CHECKPOINT_NUM}.pt"))
checkpoint.eval()

In [ ]:
predictions = []
with torch.no_grad():
    for (i, batch) in enumerate(tqdm(test_dl)):
        preds = checkpoint(to_device(batch, DEVICE))[0].numpy(force=True)
        row = test_df.iloc[i]
        predictions.append({row[f"image{j}"]: preds[j] for j in range(10)})

In [ ]:
evaluate(
    test_df.iloc[:, 2:-1].values,
    test_df["label"].values.reshape(-1, 1),
    predictions,
)

In [ ]:
# creates a file in <project root>/data with submissions in target format
with open(PATH / f"blip-{HEAD}-{MODEL_VERSION}-{CHECKPOINT_NUM}_submission.json", 'w') as f:
    json.dump([{k: str(v) for k, v in p.items()} for p in predictions], f, indent=2)